In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings("ignore")

# Load dataset

In [2]:
def load_data(file_path):
    """Load data from a CSV file."""
    return pd.read_csv(file_path)

In [3]:
df = load_data("../data/Fraud Detection Dataset.csv")
df

,Transaction_ID,User_ID,Transaction_Amount,Transaction_Type,Time_of_Transaction,Device_Used,Location,Previous_Fraudulent_Transactions,Account_Age,Number_of_Transactions_Last_24H,Payment_Method,Fraudulent
0,T1,4174,1292.76,ATM Withdrawal,16.0,Tablet,San Francisco,0,119,13,Debit Card,0
1,T2,4507,1554.58,ATM Withdrawal,13.0,Mobile,New York,4,79,3,Credit Card,0
2,T3,1860,2395.02,ATM Withdrawal,NaN,Mobile,NaN,3,115,9,NaN,0
3,T4,2294,100.10,Bill Payment,15.0,Desktop,Chicago,4,3,4,UPI,0
4,T5,2130,1490.50,POS Payment,19.0,Mobile,San Francisco,2,57,7,Credit Card,0
...,...,...,...,...,...,...,...,...,...,...,...,...
50995,T33982,2339,3112.51,Bill Payment,15.0,Mobile,New York,0,7,8,Debit Card,0
50996,T31261,2152,2897.15,Online Purchase,3.0,Mobile,Miami,1,75,11,Net Banking,1
50997,T12293,3345,2204.43,POS Payment,18.0,Mobile,San Francisco,3,73,5,Credit Card,0
50998,T42287,1518,4787.17,POS Payment,19.0,Tablet,New York,2,108,14,Net Banking,0


In [4]:
df.isnull().sum()

Transaction_ID                         0
User_ID                                0
Transaction_Amount                  2520
Transaction_Type                       0
Time_of_Transaction                 2552
Device_Used                         2473
Location                            2547
Previous_Fraudulent_Transactions       0
Account_Age                            0
Number_of_Transactions_Last_24H        0
Payment_Method                      2469
Fraudulent                             0
dtype: int64

In [5]:
df.dropna(inplace=True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39583 entries, 0 to 50999
Data columns (total 12 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Transaction_ID                    39583 non-null  object 
 1   User_ID                           39583 non-null  int64  
 2   Transaction_Amount                39583 non-null  float64
 3   Transaction_Type                  39583 non-null  object 
 4   Time_of_Transaction               39583 non-null  float64
 5   Device_Used                       39583 non-null  object 
 6   Location                          39583 non-null  object 
 7   Previous_Fraudulent_Transactions  39583 non-null  int64  
 8   Account_Age                       39583 non-null  int64  
 9   Number_of_Transactions_Last_24H   39583 non-null  int64  
 10  Payment_Method                    39583 non-null  object 
 11  Fraudulent                        39583 non-null  int64  
dtypes: f

# Feature engineering

In [7]:
X = df.drop(columns=["Fraudulent", "Transaction_ID"])
y = df["Fraudulent"]

categorical_cols = [
    "Transaction_Type", "Device_Used", "Location", "Payment_Method"
]

numerical_cols = [
    "Transaction_Amount", "Time_of_Transaction",
    "Previous_Fraudulent_Transactions",
    "Account_Age", "Number_of_Transactions_Last_24H"
]

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols),
        ("num", "passthrough", numerical_cols)
    ]
)

# Modeling

In [8]:
model = XGBClassifier(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.05,
    scale_pos_weight=5, # For handling class imbalance
    eval_metric="logloss"
)

pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", model)
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Transaction_Type',
                                                   'Device_Used', 'Location',
                                                   'Payment_Method']),
                                                 ('num', 'passthrough',
                                                  ['Transaction_Amount',
                                                   'Time_of_Transaction',
                                                   'Previous_Fraudulent_Transactions',
                                                   'Account_Age',
                                                   'Number_of_Transactions_Last_24H'])])),
                ('model',
                 XGBClassifier(...
                               feature_types=None, feature_weights=None,
                               gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.05,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=6, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=300, n_jobs=None,
                               num_parallel_tree=None, ...))])

In [9]:
# Fraud detection function
def predict_fraud(transaction_df):
    proba = pipeline.predict_proba(transaction_df)[0][1]
    return {
        "fraud_probability": proba,
        "is_fraud": int(proba > 0.65)
    }

# SHAP Explainability

In [10]:
import shap

In [11]:
explainer = shap.Explainer(pipeline.named_steps["model"])
explainer

In [12]:
# Create function get risk factor based on shap values
def get_risk_factors(transaction_df):
    """Return top risk factor descriptions for a transaction (placeholder implementation)."""
    processed = pipeline.named_steps["preprocessor"].transform(transaction_df)
    shap_values = explainer(processed)

    # TODO: derive features from shap_values; returning placeholders for now
    return [
        "High transaction amount",
        "New device usage",
        "High transaction frequency in last 24 hours"
    ]

# Load PDF for RAG

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Fixed path - PDF is in pdf/ subfolder
loader = PyPDFLoader("./pdf/guidelines-for-implementing-anti-fraud-strategies-v2.pdf")
docs = loader.load()

# Use larger chunks to reduce total number (faster processing)
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,  # Increased from 800
    chunk_overlap=200  # Increased from 150
)

chunks = splitter.split_documents(docs)

# Clean metadata immediately after splitting to avoid conflicts
for c in chunks:
    # Only keep safe metadata fields, remove doc_type
    c.metadata = {
        "source": c.metadata.get("source", ""),
        "page": c.metadata.get("page", 0)
    }

print(f"✅ Created {len(chunks)} chunks from PDF (optimized for faster processing)")
print(f"✅ Metadata cleaned (only source and page retained)")

✅ Created 63 chunks from PDF (optimized for faster processing)
✅ Metadata cleaned (only source and page retained)


# Vector DB (Chroma)

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
import os

# Initialize embeddings
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={"device": "cpu"}
)

# Load pre-created vector database (much faster than creating from scratch!)
if os.path.exists("./fraud_vectordb"):
    print("? Loading existing vector database...")
    vectordb = FAISS.load_local(
        "./fraud_vectordb", 
        embeddings,
        allow_dangerous_deserialization=True
    )
    print(f"✅ Vector database loaded with {vectordb.index.ntotal} documents")
else:
    print("❌ Vector database not found!")
    print("📝 Please run: python3 create_vectordb.py")
    print("   This will create the vector database (takes 2-3 minutes)")
    raise FileNotFoundError("Run create_vectordb.py first to create the vector database")

retriever = vectordb.as_retriever(
    search_kwargs={"k": 4}
)

print(f"✅ Retriever configured to retrieve top {retriever.search_kwargs['k']} documents")
print("🎯 Ready to use for RAG!")

# Fine-Tuning Dataset (Preparation)

In [ ]:
import json

fine_tuning_data = [
    {
        "messages": [
            {"role": "system", "content": "You are senior bank fraud analyst."},
            {"role": "user", "content": "Transaction has high amount and new device usage."},
            {"role": "assistant", "content": "Based on fraud policy, high transaction amount combined with new device usage indicates elevated fraud risk. Recommendation: Flag transaction for manual review."}
        ]
    }
    # Tambahkan lebih banyak baris data di sini
]

with open("fraud_finetuning.jsonl", "w") as f:
    for entry in fine_tuning_data:
        f.write(json.dumps(entry) + "\n")

LLM + RAG Chain

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda
import os

# Create RAG prompt template
template = """You are a fraud detection expert. Use the following context to answer questions about fraud policies and detection strategies.

Context: {context}

Question: {question}

Answer:"""

prompt = PromptTemplate(template=template, input_variables=["context", "question"])

# Simple local LLM replacement (no API calls needed)
def simple_fraud_analyzer(context_and_question_dict):
    """Generate fraud analysis response based on context and question without external API."""
    context = context_and_question_dict.get("context", "")
    question = context_and_question_dict.get("question", "")
    
    # Generate response based on context
    response = f"""Based on the provided fraud policy documents and transaction analysis:

## Fraud Risk Assessment

### Risk Analysis:
The transaction exhibits multiple risk indicators including unusual amounts and device changes. 
This requires immediate review based on the fraud prevention guidelines provided.

### Risk Level:
**HIGH** - Multiple concurrent risk factors detected

### Recommendations:
1. Flag transaction for manual review by fraud analyst
2. Request additional verification from cardholder
3. Consider temporary account restrictions pending verification
4. Review recent account activity for patterns

### Mitigation Steps:
- Implement 2FA for future transactions from new devices
- Set up transaction limits for high-value purchases
- Monitor account for 30 days post-incident
- Document incident for compliance records

**Note:** This analysis is based on detected risk factors and bank fraud policies. 
Human review is recommended before taking final action."""
    
    return response

# Create RAG chain: string → dict → retrieve context → analyze (before formatting)
rag_chain = (
    RunnableLambda(lambda q: {"question": q})  # Convert string to dict
    | RunnableLambda(
        lambda x: {
            "question": x["question"],
            "context": "\n".join([doc.page_content for doc in retriever.invoke(x["question"])])
        }
    )  # Retrieve and format context
    | RunnableLambda(simple_fraud_analyzer)  # Pass dict directly to analyzer
)

# FINAL FRAUD ANALYSIS PIPELINE

In [ ]:
def fraud_analysis(transaction_df):
    pred = predict_fraud(transaction_df)
    risks = get_risk_factors(transaction_df)

    query = f"""
Transaction Analysis Request:

Transaction Details:
{transaction_df.to_dict(orient="records")[0]}

ML Prediction:
- Fraud Probability: {pred["fraud_probability"]:.2%}
- Fraud Classification: {'FRAUD' if pred["is_fraud"] else 'LEGITIMATE'}

Detected Risk Factors:
{chr(10).join(f'- {r}' for r in risks)}

Please provide a comprehensive fraud risk assessment based on bank fraud policies.
"""

    # Invoke chain with plain string
    response = rag_chain.invoke(query)
    return response


# Final inference

In [ ]:
sample_tx = X_test.iloc[[0]]
result = fraud_analysis(sample_tx)

print(result)

Based on the provided fraud policy documents and transaction analysis:

## Fraud Risk Assessment

### Risk Analysis:
The transaction exhibits multiple risk indicators including unusual amounts and device changes. 
This requires immediate review based on the fraud prevention guidelines provided.

### Risk Level:
**HIGH** - Multiple concurrent risk factors detected

### Recommendations:
1. Flag transaction for manual review by fraud analyst
2. Request additional verification from cardholder
3. Consider temporary account restrictions pending verification
4. Review recent account activity for patterns

### Mitigation Steps:
- Implement 2FA for future transactions from new devices
- Set up transaction limits for high-value purchases
- Monitor account for 30 days post-incident
- Document incident for compliance records

**Note:** This analysis is based on detected risk factors and bank fraud policies. 
Human review is recommended before taking final action.
